In [1]:
import socket
import numpy as np
import pickle as pkl
import sys
import time
from Server import Server
from Client import Client
import matplotlib.pyplot as plt
import matplotlib

# Display plots inline and change default figure size
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
reg_lambda=0.01
iterations = 20
server = Server(20,30,10)

only use cpu

In [3]:
# SERVER_HOST = "0.0.0.0"
# SERVER_PORT = 5001

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((socket.gethostname(), 1234))
s.listen(5)

print("Server start at: %s:%s"%(socket.gethostname(), 1234))
print("wait for connection...")

conn, addr = s.accept()
print("Connected by",addr)
for i in range(iterations):
    data = b""
#     print('receiving1')
    mse_len1 = conn.recv(1024)
    mse_len1 = pkl.loads(mse_len1,encoding="bytes")
    conn.send(bytes("server received you message.","utf-8"))
    temp = conn.recv(mse_len1+1)
    data += temp
    unserialized_data1 = pkl.loads(data,encoding="bytes")
    conn.send(bytes("server received you message.","utf-8"))
    encrypt_z1, y = unserialized_data1
#     print('receiving2')
    mse_len2 = conn.recv(1024)
    mse_len2 = pkl.loads(mse_len2,encoding="bytes")
    conn.send(bytes("server received you message.","utf-8"))
    temp = conn.recv(mse_len2+1)
    data += temp
    unserialized_data2 = pkl.loads(data,encoding="bytes")
    conn.send(bytes("server received you message.","utf-8"))
    encrypt_z2, y = unserialized_data2
    
    encrypt_z = np.concatenate((encrypt_z1,encrypt_z2),axis=1)
    
#     print('forward - backward')
    server.forward(encrypt_z)
    server.cal_acc(y,i)
    server.backprop(y)

    delta2 = server.send_msg()
    serialized_data = pkl.dumps(delta2)
    mse_len = pkl.dumps(sys.getsizeof(serialized_data))

    conn.send(mse_len)

    conn.recv(1024)
    conn.send(serialized_data)

conn.close()

Server start at: DESKTOP-32B4E9T:1234
wait for connection...
Connected by ('192.168.1.161', 13866)
training acccuracy after iteration 0: 0.1
training acccuracy after iteration 1: 0.1
training acccuracy after iteration 2: 0.1
training acccuracy after iteration 3: 0.0
training acccuracy after iteration 4: 0.199
training acccuracy after iteration 5: 0.0
training acccuracy after iteration 6: 0.2
training acccuracy after iteration 7: 0.118
training acccuracy after iteration 8: 0.331
training acccuracy after iteration 9: 0.309
training acccuracy after iteration 10: 0.417
training acccuracy after iteration 11: 0.421
training acccuracy after iteration 12: 0.46
training acccuracy after iteration 13: 0.526
training acccuracy after iteration 14: 0.695
training acccuracy after iteration 15: 0.6
training acccuracy after iteration 16: 0.7
training acccuracy after iteration 17: 0.689
training acccuracy after iteration 18: 0.7
training acccuracy after iteration 19: 0.9


using GPU to train the model on the server side

tensorflow version 2.0

In [3]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(100,input_dim=100, activation=tf.nn.relu),
#         tf.keras.layers.Dense(64, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 11,110
Trainable params: 11,110
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [6]:
# SERVER_HOST = "0.0.0.0"
# SERVER_PORT = 5001

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((socket.gethostname(), 1234))
s.listen(5)

print("Server start at: %s:%s"%(socket.gethostname(), 1234))
print("wait for connection...")

conn, addr = s.accept()
print("Connected by",addr)
for i in range(iterations):
    data = b""
# client 1 data
    mse_len1 = conn.recv(1024)
    mse_len1 = pkl.loads(mse_len1,encoding="bytes")
    conn.send(bytes("server received you message.","utf-8"))
    temp = conn.recv(mse_len1+1)
    data += temp
    unserialized_data1 = pkl.loads(data,encoding="bytes")
    conn.send(bytes("server received you message.","utf-8"))
    encrypt_z1, y = unserialized_data1
# client 2 data
    mse_len2 = conn.recv(1024)
    mse_len2 = pkl.loads(mse_len2,encoding="bytes")
    conn.send(bytes("server received you message.","utf-8"))
    temp = conn.recv(mse_len2+1)
    data += temp
    unserialized_data2 = pkl.loads(data,encoding="bytes")
    conn.send(bytes("server received you message.","utf-8"))
    encrypt_z2, y = unserialized_data2
    
    encrypt_z = np.concatenate((encrypt_z1,encrypt_z2),axis=1)
    
# print('forward - backward')
    with tf.device("/gpu:0"):  
        model.fit(encrypt_z, y, epochs=1)
        inputs_va = tf.Variable(encrypt_z)
        with tf.GradientTape() as tape:
            preds = model(inputs_va)
        grads = tape.gradient(preds, inputs_va)
    delta2 = grads.numpy()

    serialized_data = pkl.dumps(delta2)
    mse_len = pkl.dumps(sys.getsizeof(serialized_data))

    conn.send(mse_len)

    conn.recv(1024)
    conn.send(serialized_data)

conn.close()

Server start at: DESKTOP-32B4E9T:1234
wait for connection...
Connected by ('192.168.1.161', 1039)
Train on 1000 samples
1000/1000 [==============================] - 1s 534us/sample - loss: 2.4603 - accuracy: 0.7080
Train on 1000 samples
1000/1000 [==============================] - 0s 58us/sample - loss: 0.0020 - accuracy: 1.0000
Train on 1000 samples
1000/1000 [==============================] - 0s 60us/sample - loss: 2.1408e-04 - accuracy: 1.0000
Train on 1000 samples
1000/1000 [==============================] - 0s 58us/sample - loss: 1.5300e-04 - accuracy: 1.0000
Train on 1000 samples
1000/1000 [==============================] - 0s 56us/sample - loss: 3.0391e-04 - accuracy: 1.0000
Train on 1000 samples
1000/1000 [==============================] - 0s 59us/sample - loss: 3.4315e-04 - accuracy: 1.0000
Train on 1000 samples
1000/1000 [==============================] - 0s 60us/sample - loss: 1.6962e-05 - accuracy: 1.0000
Train on 1000 samples
1000/1000 [==============================] - 0s